In [28]:
import os
import random
from utils import lexical, tui
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from prettytable import PrettyTable

normalizer = lexical.Preprocessing()

## Carregar copora

In [2]:
BASE_DIR = '../data/corpora'

corpus_ocultismo = []
for dir in os.listdir('{}/ocultismo'.format(BASE_DIR)):
    with open('{}/ocultismo/{}'.format(BASE_DIR, dir), 'r') as fl:
        corpus_ocultismo.append(fl.readlines())
        
corpus_tecnologia = []
for dir in os.listdir('{}/tecnologia'.format(BASE_DIR)):
    with open('{}/tecnologia/{}'.format(BASE_DIR, dir), 'r') as fl:
        corpus_tecnologia.append(fl.readlines())

## Word2Vec

In [5]:
sents_ocultismo = []
P = tui.Progress(len(corpus_ocultismo), '')

for doc in corpus_ocultismo:
    for p in doc:
        p = normalizer.lowercase(p)
        sents = normalizer.tokenize_sentences(p)
        sents = [ normalizer.remove_punctuation(s) for s in sents ]
        sents = [ normalizer.tokenize_words(w) for w in sents ]
        sents_ocultismo.extend(sents)
    P.progressStep()
    
print()

sents_tecnologia = []
P = tui.Progress(len(corpus_ocultismo), '')

for doc in corpus_tecnologia:
    for p in doc:
        p = normalizer.lowercase(p)
        sents = normalizer.tokenize_sentences(p)
        sents = [ normalizer.remove_punctuation(s) for s in sents ]
        sents = [ normalizer.tokenize_words(w) for w in sents ]
        sents_tecnologia.extend(sents)
    P.progressStep()

[============================================================] 100.0% ...


In [6]:
w2vmodel_ocultismo = Word2Vec(sents_ocultismo, size=200, window=5, min_count=3, workers=4)
w2vmodel_tecnologia = Word2Vec(sents_tecnologia, size=200, window=5, min_count=3, workers=4)

## Doc2Vec

In [29]:
all_docs = []
P = tui.Progress(len(corpus_ocultismo) + len(corpus_tecnologia), '')

for doc in corpus_ocultismo + corpus_tecnologia:
    p = ' '.join([ p.strip() for p in doc ])
    p = normalizer.lowercase(p)
    p = normalizer.remove_punctuation(p)
    tokens = normalizer.tokenize_words(p)
    all_docs.append(tokens)
    
    P.progressStep()

tagged = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(all_docs)]

In [30]:
d2vmodel = Doc2Vec(tagged, vector_size=42, window=2, min_count=1, workers=4)

## Análise

In [14]:
word_ocultismo = 'magia'
print('10 palavras mais similares à "{}" no corpus "ocultismo" '.format(word_ocultismo))
table = PrettyTable()
table.field_names = ('Palavra', 'similaridade')
for w in w2vmodel_ocultismo.wv.most_similar(word_ocultismo):
    table.add_row([ w[0], '{:0.2f}'.format(w[1]) ])
print(table)

print()

word_tecnologia = 'notebook'
print('10 palavras mais similares à "{}" no corpus "tecnologia" '.format(word_tecnologia))
table = PrettyTable()
table.field_names = ('Palavra', 'similaridade')
for w in w2vmodel_tecnologia.wv.most_similar(word_tecnologia):
    table.add_row([ w[0], '{:0.2f}'.format(w[1]) ])
print(table)

10 palavras mais similares à "magia" no corpus "ocultismo" 
+----------------+--------------+
|    Palavra     | similaridade |
+----------------+--------------+
|     jihad      |     0.91     |
|     missa      |     0.82     |
| indeterminação |     0.80     |
|   eucaristia   |     0.79     |
|    bruxaria    |     0.78     |
|    umbanda     |     0.78     |
|  metassistema  |     0.78     |
|     teoria     |     0.77     |
|     abelha     |     0.77     |
|      lei       |     0.77     |
+----------------+--------------+

10 palavras mais similares à "notebook" no corpus "tecnologia" 
+------------+--------------+
|  Palavra   | similaridade |
+------------+--------------+
| computador |     0.99     |
|    novo    |     0.99     |
|     o      |     0.99     |
|  teclado   |     0.99     |
|    ele     |     0.99     |
|    seu     |     0.98     |
|  parrudo   |     0.98     |
|   mesmo    |     0.98     |
|   mouse    |     0.98     |
|    este    |     0.98     |
+--------

O resultados poderiam melhorar se fosse usados textos de uma única fonte, ou de fontes mais parecidas, já que elas poderiam usar a mesma palavra com sentidos diferentes e em outros contexos resultando em um nível maior de ruído e aparentes contradições.

### Comprando corpus com Word2Vec

Um método para comprar dois corpus seria analisando a semelhança entre dois vetores de palavras, qunato mais palavras tiverem em comum, maior a chance dos dois serem similares

In [27]:
word = 'processar'

table = PrettyTable()
table.field_names = ('Palavra tec', 'similaridade tec', 'Palavra ocul', 'similaridade ocul')

l_tec = list(w2vmodel_tecnologia.wv.most_similar(word))
l_ocul = list(w2vmodel_ocultismo.wv.most_similar(word))
for i in range(10):
    table.add_row([
        l_tec[i][0], '{:0.2f}'.format(l_tec[i][1]),
        l_ocul[i][0],'{:0.2f}'.format(l_ocul[i][1])
    ])

print('Similidaridas com a palavra "{}"'.format(word))
print(table)

Similidaridas com a palavra "processar"
+-------------+------------------+------------------+-------------------+
| Palavra tec | similaridade tec |   Palavra ocul   | similaridade ocul |
+-------------+------------------+------------------+-------------------+
|   palavras  |       0.99       |       teus       |        0.99       |
|    móveis   |       0.99       |   pouquíssimas   |        0.99       |
|  conseguem  |       0.99       |    abandonar     |        0.98       |
|    jensen   |       0.99       |    atividades    |        0.98       |
|    letra    |       0.99       |      ondas       |        0.98       |
|  continuam  |       0.99       |      online      |        0.98       |
|   instinct  |       0.99       | automaticamente  |        0.98       |
|    redes    |       0.99       | superficialmente |        0.98       |
|    gsync    |       0.99       |    saturnália    |        0.98       |
|    tendo    |       0.99       |     batendo      |        0.98       

### Comparando corpus com Doc2Vec

In [37]:
docs = [
    # corpus ocultismo
    d2vmodel.infer_vector(all_docs[random.randint(0, 400)]),
    d2vmodel.infer_vector(all_docs[random.randint(0, 400)]),
    # corpus tecnologia
    d2vmodel.infer_vector(all_docs[-random.randint(0, 400)]),
    d2vmodel.infer_vector(all_docs[-random.randint(0, 400)]),
]

In [41]:
from scipy import spatial

print(1 - spatial.distance.cosine(docs[0], docs[1]))
print(1 - spatial.distance.cosine(docs[2], docs[3]))

print()

print(1 - spatial.distance.cosine(docs[0], docs[2]))
print(1 - spatial.distance.cosine(docs[0], docs[3]))
print(1 - spatial.distance.cosine(docs[1], docs[2]))
print(1 - spatial.distance.cosine(docs[1], docs[3]))

0.8875948786735535
0.9453807473182678

0.6239517331123352
0.685464084148407
0.6296059489250183
0.6872815489768982


Pode se observar que o grau de similaridade dos documentos de um mesmo corpus é significativo, acima de 88. Como foi usado fontes para compilar o primeiro corpus o nível de similaridade é menor, isso pode ser atribuido ao diferentes estilos de escrita. Para se obter um nível maior poderia dividir o corpus, para diminuir a diferença entre os estilos de escrita